# Models from Scratch: K-Nearest Neighbors
## *Implementation*
***

In [384]:
import math
import pandas as pd
import numpy as np

In [385]:
class KNN:
    
    def __init__(self, X, y, k=5):
        """
        Initalizes a KNN object
        
        Parameters: 
            X -- Predictor matrix (Pandas dataframe, numpy array, 2D list)
            y -- Vector of response variable (Pandas series, numpy array, list)
        """
        self.X = np.asarray(X)
        self.y = np.asarray(y).reshape(len(X),)
        
        self.n = len(X)
        self.p = X.shape[1]
        self.k = k
        
    def classify(self, x):
        """Simple, brute-force implementation of KNN. Classifies a feature vector x. """
        if len(x) != self.p:
            raise ValueError("Dimension Error: feature vector x has dimension {} but model has {} predictors".format(len(x), self.p))
        
        distances = []
        for index, observation in enumerate(self.X):
            d = KNN.distance(x, list(observation))
            # Append distance along with this observation's index position
            distances.append((d, index)) 
        
        # Sort distances by the first tuple value (i.e., the distance -- not the index)
        # And select only the closest k points
        distances = sorted(distances, key=lambda x: x[0])[1:self.k+1]
        
        # Use indexes to determine which y value these closest X values correspond to
        indexes = [i[1] for i in distances]
        classes = [self.y[i] for i in indexes]
        
        return KNN._most_common(classes)
    
    def train_error(self):
        "Returns the training error for this model"
        incorrect = 0
        for index, observation in enumerate(self.X):
            if self.classify(observation) != self.y[index]:
                incorrect += 1
        return incorrect / self.n
    
    def test_error(self, X, y):
        """Returns the testing error of this model given new data X and y"""
        X = np.asarray(X)
        y = np.asarray(y)
        
        incorrect = 0
        for index, observation in enumerate(X):
            if self.classify(observation) != y[index]:
                incorrect += 1
        
        return incorrect / len(X)
        
    
    # ---------- Helper methods ---------- #
    
    @staticmethod
    def distance(p1, p2):
        """Returns the euclidean distance between points p1 and p2 in p-dimensional space."""
        
        # Points must be of same dimension
        if len(p1) != len(p2):
            raise ValueError("Dimension Error: {} != {}".format(len(p1), len(p2)))
        
        distance = 0
        for pair in zip(p1, p2):
            distance += (pair[0] - pair[1])**2
        return math.sqrt(distance) 
    
    
    @staticmethod    
    def _most_common(lst):
        """Returns the most common element in a list"""
        return max(set(lst), key=lst.count)

## *Application*
***

We'll use the famous Iris dataset and apply the KNN algorithm to classify the plant species

In [386]:
df = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', names=['slength', 'swidth','plength', 'pwidth', 'species'])
# Shuffle data
df = df.sample(frac=1)
df.head()

,slength,swidth,plength,pwidth,species
66,5.6,3.0,4.5,1.5,Iris-versicolor
63,6.1,2.9,4.7,1.4,Iris-versicolor
37,4.9,3.1,1.5,0.1,Iris-setosa
18,5.7,3.8,1.7,0.3,Iris-setosa
44,5.1,3.8,1.9,0.4,Iris-setosa


In [387]:
# Predictor matrix
X = df[['slength', 'swidth','plength', 'pwidth']]

# Respones vector
y = df[['species']]

In [388]:
# Partition data into train/test sets
train_X, train_y = X[0:100], y[0:100]
test_X, test_y = X[100:], y[100:]

# Construct model
model = KNN(train_X, train_y, k=10)

# Classify a plant with the following features: 5.1, 3.5, 1.4, 0.2
model.classify([5.1, 3.5, 1.4, 0.2])

'Iris-setosa'

In [389]:
# Test the model
model.test_error(test_X, test_y)

0.04

Thus, KNN with k=10 has an accuracy of about 94%